# 여기어때 리뷰 전처리

### hanspell 체크

In [1]:
from hanspell import spell_checker

In [2]:
sent = "외않되"

spelled_sent = spell_checker.check(sent)
print(spelled_sent)

Checked(result=True, original='외않되', checked='왜 안돼', errors=1, words=OrderedDict([('왜', 1), ('안돼', 1)]), time=0.0739293098449707)


In [3]:
print(spelled_sent.checked)

왜 안돼


## 데이터 불러오기

In [4]:
import pandas as pd 
import numpy as np 

In [8]:
df = pd.read_csv('data/여기어때_hotel_review170th.csv')
df.head()

,호텔명,리뷰,별점
0,그랜드 인터컨티넨탈 서울 파르나스,제주에서 곧바로 날라와서 짐 놓고 업계 사람들 만나러 가는 약속 때문에 잡은 호텔 ...,9.6
1,그랜드 인터컨티넨탈 서울 파르나스,"친절하고, 입지는 좋습니다.허나 특급호텔 치고 청결상태가 매우 불량해 불쾌했네요.핸...",8.3
2,그랜드 인터컨티넨탈 서울 파르나스,7월3일 ~ 7월4일 킹주니어스위트 30시간 패키지 후기1. 체크인 후 짐 이동 서...,10.0
3,그랜드 인터컨티넨탈 서울 파르나스,한달만에 다시 방문했어요!저번 방문이 너무 편하고 좋아서 이번엔 동생과 함께 서울 ...,10.0
4,그랜드 인터컨티넨탈 서울 파르나스,생일 기념으로 호캉스 다녀왔습니다~ 파르나스는 처음 방문이였는데 너무너무 만족스러운...,10.0


In [9]:
# 중복 데이터 확인
df.리뷰.nunique(), len(df)

(47119, 49065)

In [10]:
# 중복 데이터 제거
df.drop_duplicates(subset=['리뷰'],inplace = True)
df.shape

(47120, 3)

In [11]:
df.shape

(47120, 3)

In [12]:
# null데이터 확인
df.isnull().sum()

호텔명    0
리뷰     1
별점     0
dtype: int64

In [13]:
df.fillna(" ",inplace=True) # dropna가 오류나서 일단 fillna
df.isnull().sum()

호텔명    0
리뷰     0
별점     0
dtype: int64

### 텍스트 전처리
한글과 공백 이외에는 제거

In [14]:
df['리뷰'] = df.리뷰.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
df.head(10)

<ipython-input-14-af2e31446549>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['리뷰'] = df.리뷰.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')


,호텔명,리뷰,별점
0,그랜드 인터컨티넨탈 서울 파르나스,제주에서 곧바로 날라와서 짐 놓고 업계 사람들 만나러 가는 약속 때문에 잡은 호텔 ...,9.6
1,그랜드 인터컨티넨탈 서울 파르나스,친절하고 입지는 좋습니다허나 특급호텔 치고 청결상태가 매우 불량해 불쾌했네요핸드타올...,8.3
2,그랜드 인터컨티넨탈 서울 파르나스,월일 월일 킹주니어스위트 시간 패키지 후기 체크인 후 짐 이동 서비스는 가급적 받...,10.0
3,그랜드 인터컨티넨탈 서울 파르나스,한달만에 다시 방문했어요저번 방문이 너무 편하고 좋아서 이번엔 동생과 함께 서울 나...,10.0
4,그랜드 인터컨티넨탈 서울 파르나스,생일 기념으로 호캉스 다녀왔습니다 파르나스는 처음 방문이였는데 너무너무 만족스러운 ...,10.0
5,그랜드 인터컨티넨탈 서울 파르나스,너무너무좋아영 조식이 정말 맛있네요 ㅋㅋㅋㅋㅋ,10.0
6,그랜드 인터컨티넨탈 서울 파르나스,룸 컨디션도 넘 좋고 편하고 조용하게 잘 쉬다 갑니다,10.0
7,그랜드 인터컨티넨탈 서울 파르나스,높기만 했어요룸업글되어 좋긴했습니당,8.6
8,그랜드 인터컨티넨탈 서울 파르나스,먼저 리노베이션을 했기 때문에 기본적으로깨끗하긴 합니다그러나 별점을 많이 깎은건 하...,2.0
9,그랜드 인터컨티넨탈 서울 파르나스,친절한 서비스와 가격대비 효율 좋아요,10.0


공백만 남은 데이터는 Nan으로 변환한 후 제거

In [15]:
df.리뷰.replace('',np.nan, inplace=True)
df.리뷰.replace(' ',np.nan, inplace=True)
df.리뷰.isnull().sum()

11

In [17]:
df = df.dropna(how='any')
df.shape

(47067, 3)

ㅎ ㅋ ㅠ 등 자모음 하나씩만 있는 것 제거 & 맞춤법 검사기 돌리기

In [21]:
import re

In [22]:
df['length']  = df.리뷰.apply(lambda x : len(x))
df.length

0        341
1        141
2        521
3        286
4        196
        ... 
49060     76
49061    133
49062     46
49063     34
49064     71
Name: length, Length: 47067, dtype: int64

In [23]:
df.length.sort_values()

13950       1
3285        1
3333        1
3146        1
46786       1
         ... 
26525    2133
13563    2218
34662    2407
10584    4624
8853     4730
Name: length, Length: 47067, dtype: int64

In [17]:
# 0인 애들이 있으면 삭제
'''zero = df[df.length == 0].index
df = df.drop(zero)
df.length'''

0        341
1        141
2        521
3        286
4        196
        ... 
99900     41
99901     20
99902     33
99903     26
99904      8
Name: length, Length: 93219, dtype: int64

In [24]:
# 인덱스 재배열
df.reset_index(drop=True, inplace=True)

In [25]:
df.length.sort_values()

13377       1
3210        1
3256        1
3076        1
44885       1
         ... 
25351    2133
12998    2218
33295    2407
10148    4624
8452     4730
Name: length, Length: 47067, dtype: int64

In [26]:
for i in range(0, len(df)) :
    df.loc[i,'정제1'] = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',df.loc[i]['리뷰'])

In [29]:
df.loc[0].리뷰

'제주에서 곧바로 날라와서 짐 놓고 업계 사람들 만나러 가는 약속 때문에 잡은 호텔 입니다 룸 컨디션이 매우 좋고 룸 어메니티는 조말론을 씁니다 프론트 분들도 친절하시고 역시가 역시다 할 정도로 명불허전 입니다 다음날 로비 라운지에서도 명 먹을 에프터눈 인의 추천에도 그냥 인으로 먹을수 있을 정도로 매우 친절했습니다 단 잠시 잊고 있었던 약속 지키려 케리어를 컨시어지에 맡겼는데 케리어 다시 찾으러 갈때 컨시어지 분은 약간 불친절 하였습니다 마무리에 약간 기분이 상했는데 다음에 갔을땐 이런일이 없겠죠 개선이 되어 있을거라고 희망해 봅니다 ㅎㅎ여튼 다시 찾아와 돈을 써도 전혀 아깝지 않은 호텔 중 하나 입니다'

In [27]:
df.loc[0].정제1

'제주에서 곧바로 날라와서 짐 놓고 업계 사람들 만나러 가는 약속 때문에 잡은 호텔 입니다 룸 컨디션이 매우 좋고 룸 어메니티는 조말론을 씁니다 프론트 분들도 친절하시고 역시가 역시다 할 정도로 명불허전 입니다 다음날 로비 라운지에서도 명 먹을 에프터눈 인의 추천에도 그냥 인으로 먹을수 있을 정도로 매우 친절했습니다 단 잠시 잊고 있었던 약속 지키려 케리어를 컨시어지에 맡겼는데 케리어 다시 찾으러 갈때 컨시어지 분은 약간 불친절 하였습니다 마무리에 약간 기분이 상했는데 다음에 갔을땐 이런일이 없겠죠 개선이 되어 있을거라고 희망해 봅니다 여튼 다시 찾아와 돈을 써도 전혀 아깝지 않은 호텔 중 하나 입니다'

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47067 entries, 0 to 47066
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   호텔명     47067 non-null  object 
 1   리뷰      47067 non-null  object 
 2   별점      47067 non-null  float64
 3   length  47067 non-null  int64  
 4   정제1     47067 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.8+ MB


In [31]:
df.to_csv('data/여기어때_170까지_전처리1단계.csv',encoding='utf-8-sig')

In [32]:
for i in range(0, len(df) + 1) :
    if df.loc[i].length <= 500 : 
        try : 
            refined = spell_checker.check(df.loc[i]['정제1']).checked
        except : 
            refined = df.loc[i]['정제1']
    else : 
        refined = spell_checker.check(df.loc[i,'정제1'][:500]).checked

    df.loc[i,'정제_리뷰'] = refined
    print(i)

3
43734
43735
43736
43737
43738
43739
43740
43741
43742
43743
43744
43745
43746
43747
43748
43749
43750
43751
43752
43753
43754
43755
43756
43757
43758
43759
43760
43761
43762
43763
43764
43765
43766
43767
43768
43769
43770
43771
43772
43773
43774
43775
43776
43777
43778
43779
43780
43781
43782
43783
43784
43785
43786
43787
43788
43789
43790
43791
43792
43793
43794
43795
43796
43797
43798
43799
43800
43801
43802
43803
43804
43805
43806
43807
43808
43809
43810
43811
43812
43813
43814
43815
43816
43817
43818
43819
43820
43821
43822
43823
43824
43825
43826
43827
43828
43829
43830
43831
43832
43833
43834
43835
43836
43837
43838
43839
43840
43841
43842
43843
43844
43845
43846
43847
43848
43849
43850
43851
43852
43853
43854
43855
43856
43857
43858
43859
43860
43861
43862
43863
43864
43865
43866
43867
43868
43869
43870
43871
43872
43873
43874
43875
43876
43877
43878
43879
43880
43881
43882
43883
43884
43885
43886
43887
43888
43889
43890
43891
43892
43893
43894
43895
43896
43897
43898
43899
43

KeyError: 47067

In [35]:
df.to_csv('data/여기어때_전처리_170까지.csv',encoding='utf-8-sig')

- 아고다, 야놀자, 여기어때 모두 합쳐서 호텔별 리뷰수 확인해서 적은 갯수를 가진 것들을 제거
        글자수 10개 미만인 것 제거
        

In [16]:
import pandas as pd 
import numpy as np
df = pd.read_csv('data/여기어때_전처리_171부터.csv')
df.head()

,Unnamed: 0,호텔명,리뷰,별점,length,정제1,정제_리뷰
0,0,호텔 파이 종로,진짜 가성비 좋아요침대 시트 전체적인 환경이 매우 쾌적해요화장실도 깔끔합니다다만 수...,10.0,79,진짜 가성비 좋아요침대 시트 전체적인 환경이 매우 쾌적해요화장실도 깔끔합니다다만 수...,진짜 구성비 좋아요 침대 시트 전체적인 환경이 매우 쾌적해요 화장실도 깔끔합니다 다...
1,1,호텔 파이 종로,좀 좁긴 했는데 깔끔하고 만족스러웠습니다 사장님이 친절하세요 ㅎㅎ,9.3,37,좀 좁긴 했는데 깔끔하고 만족스러웠습니다 사장님이 친절하세요,좀 좁긴 했는데 깔끔하고 만족스러웠습니다 사장님이 친절하세요
2,2,호텔 파이 종로,직원분도 친절하시고 깨끗해서 다음에 또 묵어야겠다고 생각한 곳이었습니다,10.0,39,직원분도 친절하시고 깨끗해서 다음에 또 묵어야겠다고 생각한 곳이었습니다,직원분도 친절하시고 깨끗해서 다음에 또 묵어야겠다고 생각한 곳이었습니다
3,3,호텔 파이 종로,주차장이 많이 협소합니다ㅠㅠ,10.0,15,주차장이 많이 협소합니다,주차장이 많이 협소합니다
4,4,호텔 파이 종로,깨끗하고 아늑한 가성비좋은 숙소,10.0,17,깨끗하고 아늑한 가성비좋은 숙소,깨끗하고 아늑한 구성비 좋은 숙소


In [17]:
df.정제_리뷰.replace('',np.nan, inplace=True)
df.정제_리뷰.replace(' ',np.nan, inplace=True)
df.정제_리뷰.isnull().sum()

11

In [18]:
df = df.dropna(how='any')

In [19]:
df['length2']  = df.정제_리뷰.apply(lambda x : len(x))
df.length2.sort_values()

32114      1
8884       1
2098       1
18193      1
32036      1
        ... 
7497     579
35981    580
24899    584
41455    602
19900    678
Name: length2, Length: 46861, dtype: int64

- 열글자 이하 리뷰 삭제

In [20]:
ten = df[df.length2 <= 10].index
df = df.drop(ten)
df.length2

0         84
1         35
2         39
3         13
4         18
        ... 
46866     62
46867     31
46868     27
46869    197
46870     57
Name: length2, Length: 43896, dtype: int64

In [21]:
df.length2.sort_values()

16894     11
43350     11
43357     11
46504     11
35597     11
        ... 
7497     579
35981    580
24899    584
41455    602
19900    678
Name: length2, Length: 43896, dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43896 entries, 0 to 46870
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  43896 non-null  int64  
 1   호텔명         43896 non-null  object 
 2   리뷰          43896 non-null  object 
 3   별점          43896 non-null  float64
 4   length      43896 non-null  int64  
 5   정제1         43896 non-null  object 
 6   정제_리뷰       43896 non-null  object 
 7   length2     43896 non-null  int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 3.0+ MB


In [23]:
# 야놀자 점수체계와 맞추기
df.별점 = round(df.별점/2,0)

In [24]:
df.별점

0        5.0
1        5.0
2        5.0
3        5.0
4        5.0
        ... 
46866    5.0
46867    4.0
46868    5.0
46869    5.0
46870    5.0
Name: 별점, Length: 43896, dtype: float64

In [25]:
df.to_csv('data/여기어때_전처리_171부터_최종.csv',encoding='utf-8-sig')